# Week 3 Assignment

Segmenting and Clustering Neighborhoods in Toronto

In [0]:
# Load Packages
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [2]:
# request the webpage first
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
re = requests.get(url)

# read the webpage as soup
soup = BeautifulSoup(re.text)

# Find the table 'wikitable sortable jquery-tablesorter'
table = soup.find('table')
table_dic = {}
# load the table headers
headers = table.find_all('th')
for head in headers:
  head_text = head.text.split('\n')[0]
  table_dic[head_text] = []
  
table_dic

{'Borough': [], 'Neighbourhood': [], 'Postcode': []}

In [0]:
# Append values into the dictionary
for i,index in enumerate(table.find_all('td')):
  if (i+1)%3 == 1:
    table_dic['Postcode'].append(index.text.split('\n')[0])
  elif (i+1)%3 == 2:
    table_dic['Borough'].append(index.text.split('\n')[0])
  elif (i+1)%3 == 0:
    table_dic['Neighbourhood'].append(index.text.split('\n')[0])

In [4]:
# load the dictionary to a pandas dataframe
dat = pd.DataFrame.from_dict(table_dic)
dat.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


To create the above dataframe:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
# 2. filt borough
dat1 = dat.loc[dat['Borough']!='Not assigned',]
# 3. combine neighborhoods
dat2 = dat1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ",".join(x)).reset_index(name='Neighbourhood')
# 4. clean neighborhood
dat2.loc[dat2['Neighbourhood']=='Not assigned','Neighbourhood'] = dat2.loc[dat2['Neighbourhood']=='Not assigned','Borough']
dat2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Print the number of rows of the dataframe

In [6]:
print('Number of rows is',dat2.shape[0])

Number of rows is 103


## Add the latitude and longitude

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:



In [0]:
!pip install geocoder

In [0]:
import geocoder # import geocoder
postal_code = 'M5G'

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print('The latitude and longitude are (',latitude,', ',longitude,')')

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [7]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
file = '/content/drive/My Drive/Colab Notebooks/Coursera/IBM Data Science Capstone/Geospatial_Coordinates.csv'
locate_dat = pd.read_csv(file)
locate_dat = locate_dat.rename(columns={'Postal Code': "Postcode"})
locate_dat.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
dat3 = pd.merge(dat2, locate_dat, how='left', on='Postcode')
dat3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Segmentation and Clustering

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.

2. to generate maps to visualize your neighborhoods and how they cluster together.

In [10]:
borough_values = dat3['Borough'].values
toronto_idxs = [i for i,b in enumerate(borough_values) if 'Toronto' in b]
dat_toronto = dat3.iloc[toronto_idxs]
dat_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [11]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

# Credentials for Foursquare
CLIENT_ID = 'RLGSBQG4IUCCFW42BJXRA2WG5SFJCF1F2O5IB50C3XIITQHW' # your Foursquare ID
CLIENT_SECRET = '3MIND2EJZV0MTUCGJHVS13YUZH0YVLGHPIH5ZI5HTZ2CF0IJ' # your Foursquare Secret
VERSION = '20190915'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RLGSBQG4IUCCFW42BJXRA2WG5SFJCF1F2O5IB50C3XIITQHW
CLIENT_SECRET:3MIND2EJZV0MTUCGJHVS13YUZH0YVLGHPIH5ZI5HTZ2CF0IJ


In [0]:
# Function used to get all nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=dat_toronto['Neighbourhood'],
                                   latitudes=dat_toronto['Latitude'],
                                   longitudes=dat_toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [14]:
# Head of the dataset
print(toronto_venues.shape)
toronto_venues.head()

(1702, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [32]:
# how many venues were returned for each neighborhood
d = toronto_venues.groupby('Neighborhood').count().reset_index('Neighborhood')
d[['Neighborhood','Venue']]

,Neighborhood,Venue
0,"Adelaide,King,Richmond",100
1,Berczy Park,55
2,"Brockton,Exhibition Place,Parkdale Village",22
3,Business Reply Mail Processing Centre 969 Eastern,16
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",16
5,"Cabbagetown,St. James Town",46
6,Central Bay Street,84
7,"Chinatown,Grange Park,Kensington Market",100
8,Christie,16
9,Church and Wellesley,90


### Analyze Each Neighborhood

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
cols = list(toronto_onehot.columns)
n_idx = cols.index('Neighborhood')
cols[:n_idx] + cols[(n_idx+1):]
# move neighborhood column to the first column
fixed_columns = [cols[n_idx]] + cols[:n_idx] + cols[(n_idx+1):]
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1702, 239)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(38, 239)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.03,0.0,0.0,0.010000,0.01,0.0,0.02,0.0,0.0000,0.000000,0.0,0.000000,0.020000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.01,0.0000,0.01,0.020000,0.0000,0.0,0.01,0.03,...,0.010000,0.0,0.000000,0.0000,0.0,0.0100,0.0,0.0,0.0,0.0,0.0000,0.01,0.0,0.0,0.000000,0.0,0.040000,0.0,0.0,0.02,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.030000,0.01,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.000000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.018182,0.00,0.0,0.00,0.0,0.0000,0.018182,0.0,0.018182,0.036364,0.0,0.000000,0.0,0.018182,0.018182,0.0,0.036364,0.0,0.018182,0.018182,0.0000,0.00,0.0000,0.00,0.018182,0.0000,0.0,0.00,0.00,...,0.036364,0.0,0.018182,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.000000,0.0,0.036364,0.0,0.0,0.00,0.0,0.0,0.018182,0.0,0.0,0.0,0.018182,0.0,0.018182,0.00,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.00,0.0,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0000,0.000000,0.0,0.000000,0.045455,0.0,0.045455,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.00,0.0000,0.00,0.090909,0.0000,0.0,0.00,0.00,...,0.000000,0.0,0.000000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.045455,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0625,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.00,0.0000,0.00,0.000000,0.0625,0.0,0.00,0.00,...,0.000000,0.0,0.000000,0.0625,0.0,0.0625,0.0,0.0,0.0,0.0,0.0625,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.062500,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0625,0.00,0.0625,0.00,0.000000,0.0000,0.0,0.00,0.00,...,0.000000,0.0,0.000000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


Print each neighborhood along with the top 5 most common venues

In [97]:
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Steakhouse  0.04
3                  Bar  0.04
4  American Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1     Coffee Shop  0.09
2            Café  0.09
3          Bakery  0.05
4       Pet Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1             Brewery  0.06
2          Skate Park  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3

In [112]:
# Store into the dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# create the new dataframe and display the top 5 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Restaurant,Hotel,Burger Joint,Cosmetics Shop,Thai Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Steakhouse,Beer Bar,Cheese Shop,Farmers Market,Café,Bakery,Italian Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Intersection,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Garden,Restaurant,Burrito Place,Brewery,Skate Park,Farmers Market,Smoke Shop,Fast Food Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Plane,Sculpture Garden,Coffee Shop,Boat or Ferry,Harbor / Marina,Boutique,Airport Gate


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 3 clusters.

In [113]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [114]:
## create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dat_toronto.rename(columns={'Neighbourhood': "Neighborhood"})

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Trail,Pub,Discount Store,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Brewery,Bubble Tea Shop,Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Liquor Store,Movie Theater,Burger Joint,Fast Food Restaurant,Sandwich Place,Burrito Place,Fish & Chips Shop,Steakhouse,Italian Restaurant,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Latin American Restaurant,Bookstore,Brewery
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Visualize the resulting clusters

In [117]:
# create map
latitude = 43.651070
longitude = -79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [118]:
## Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,0,Park,Bus Line,Swim School,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
48,Central Toronto,0,Park,Tennis Court,Yoga Studio,Discount Store,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
50,Downtown Toronto,0,Park,Playground,Trail,Building,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [119]:
## Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,1,Health Food Store,Trail,Pub,Discount Store,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
41,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Brewery,Bubble Tea Shop,Restaurant
42,East Toronto,1,Liquor Store,Movie Theater,Burger Joint,Fast Food Restaurant,Sandwich Place,Burrito Place,Fish & Chips Shop,Steakhouse,Italian Restaurant,Sushi Restaurant
43,East Toronto,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Latin American Restaurant,Bookstore,Brewery
45,Central Toronto,1,Breakfast Spot,Gym,Park,Hotel,Sandwich Place,Asian Restaurant,Food & Drink Shop,Clothing Store,Yoga Studio,Eastern European Restaurant
46,Central Toronto,1,Clothing Store,Sporting Goods Shop,Coffee Shop,Café,Shoe Store,Burger Joint,Salon / Barbershop,Restaurant,Rental Car Location,Yoga Studio
47,Central Toronto,1,Dessert Shop,Sandwich Place,Gym,Pizza Place,Café,Italian Restaurant,Coffee Shop,Sushi Restaurant,Toy / Game Store,Park
49,Central Toronto,1,Coffee Shop,Pub,Fried Chicken Joint,Light Rail Station,Restaurant,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Liquor Store
51,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Pizza Place,Italian Restaurant,Bakery,Pub,Breakfast Spot,Caribbean Restaurant,Market
52,Downtown Toronto,1,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Hotel,Gastropub,Burger Joint,Café,Pub


In [120]:
## Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Garden,Home Service,Pool,Yoga Studio,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
